# Data Collection
Phase 2: S&P 500 대규모 데이터 수집

**목표:**
- S&P 500 전체 종목 (503개) 데이터 수집
- 가격 데이터: 10년치
- 재무제표: 연간/분기
- 종목별 데이터 시작 연도 저장 (동적 필터링용)

**예상 소요 시간:** 30~50분

## 1. 환경 설정

In [17]:
import sys
import os
import json

# === 경로 설정 ===
# 소스 코드 경로 (LongArc)
backend_path = '/Users/kuka/LongArc/backend'
if backend_path not in sys.path:
    sys.path.insert(0, backend_path)

# 기존 데이터 로드 경로 (LongArc - 이미 수집된 데이터)
source_dir = '/Users/kuka/LongArc/data/raw'

# 결과 저장 경로 (5X Finder - 새 프로젝트)
output_dir = '/Users/kuka/5X Finder/data/raw'
os.makedirs(output_dir, exist_ok=True)

print(f"Backend 경로: {backend_path}")
print(f"데이터 로드 경로: {source_dir}")
print(f"결과 저장 경로: {output_dir}")

Backend 경로: /Users/kuka/LongArc/backend
데이터 로드 경로: /Users/kuka/LongArc/data/raw
결과 저장 경로: /Users/kuka/5X Finder/data/raw


In [18]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from datetime import datetime
import time
import warnings
warnings.filterwarnings('ignore')

from data.collector import DataCollector
from data.universe import Universe
from features.calculator import FeatureCalculator

print("✅ 라이브러리 로드 완료")

✅ 라이브러리 로드 완료


## 2. 유니버스 로드

In [19]:
from data.universe import Universe
from data.collector import DataCollector

universe = Universe()
collector = DataCollector()

# S&P 500 종목 로드
sp500 = universe.get_sp500_tickers()
print(f"S&P 500: {len(sp500)}개")
print(f"처음 10개: {sp500[:10]}")

S&P 500: 503개
처음 10개: ['MMM', 'AOS', 'ABT', 'ABBV', 'ACN', 'ADBE', 'AMD', 'AES', 'AFL', 'A']


## 3. 종목별 데이터 범위 스캔

### 왜 하는가?
**동적 필터링**을 위해 각 종목의 데이터 시작 연도를 알아야 함.
- META: 2012년 상장 → 2012년부터 사용 가능
- UBER: 2019년 상장 → 2019년부터 사용 가능
- AAPL: 1980년 상장 → 2010년부터 사용 가능

In [20]:
# 기존 스캔 결과 로드 (LongArc에서)
start_years_path = os.path.join(source_dir, 'ticker_start_years.json')

if os.path.exists(start_years_path):
    with open(start_years_path, 'r') as f:
        ticker_start_years = json.load(f)
    print(f"✅ 기존 스캔 결과 로드: {len(ticker_start_years)}개 종목")
    skip_scan = True
else:
    print("스캔 결과 없음. 새로 스캔 시작...")
    skip_scan = False

✅ 기존 스캔 결과 로드: 502개 종목


In [21]:
# 종목별 데이터 시작일 스캔 (skip_scan=False일 때만 실행)
if not skip_scan:
    import yfinance as yf
    print(f"총 종목 수: {len(sp500)}개")
    
    data_range = []
    
    for ticker in tqdm(sp500, desc="데이터 범위 스캔"):
        try:
            stock = yf.Ticker(ticker)
            hist = stock.history(period="max")
            
            if len(hist) > 0:
                data_range.append({
                    'ticker': ticker,
                    'start_date': hist.index[0],
                    'end_date': hist.index[-1],
                    'years': (hist.index[-1] - hist.index[0]).days / 365
                })
        except Exception as e:
            print(f"{ticker} 실패: {e}")
    
    range_df = pd.DataFrame(data_range)
    
    # 종목별 시작 연도 추출
    range_df['start_year'] = pd.to_datetime(range_df['start_date']).dt.year
    ticker_start_years = dict(zip(range_df['ticker'], range_df['start_year']))
    
    print(f"\n✅ 스캔 완료: {len(ticker_start_years)}개 종목")

In [22]:
# 연도별 사용 가능 종목 수 확인
print("=== 연도별 사용 가능 종목 수 ===")
for year in range(2010, 2020):
    available = [t for t, start in ticker_start_years.items() if start <= year]
    print(f"{year}년: {len(available)}개 종목")

=== 연도별 사용 가능 종목 수 ===
2010년: 435개 종목
2011년: 442개 종목
2012년: 451개 종목
2013년: 460개 종목
2014년: 466개 종목
2015년: 471개 종목
2016년: 477개 종목
2017년: 479개 종목
2018년: 482개 종목
2019년: 489개 종목


In [23]:
# 2010년 이후 상장 종목 확인
recent_ipos = [(t, y) for t, y in ticker_start_years.items() if y >= 2010]
recent_ipos = sorted(recent_ipos, key=lambda x: x[1])

print(f"\n=== 2010년 이후 상장 종목 ({len(recent_ipos)}개) ===")
for ticker, year in recent_ipos[:15]:
    print(f"  {ticker}: {year}년")
if len(recent_ipos) > 15:
    print(f"  ... 외 {len(recent_ipos) - 15}개")


=== 2010년 이후 상장 종목 (77개) ===
  CBOE: 2010년
  CHTR: 2010년
  CPAY: 2010년
  GNRC: 2010년
  GM: 2010년
  KKR: 2010년
  LYB: 2010년
  NXPI: 2010년
  TRGP: 2010년
  TSLA: 2010년
  APO: 2011년
  APTV: 2011년
  HCA: 2011년
  HII: 2011년
  KMI: 2011년
  ... 외 62개


## 4. 이미 수집된 데이터 확인

In [24]:
# 이미 수집된 종목 확인 (LongArc에서)
prices_dir = os.path.join(source_dir, 'prices')
financials_dir = os.path.join(source_dir, 'financials')

# 수집된 가격 데이터
collected_prices = set()
if os.path.exists(prices_dir):
    collected_prices = {f.replace('.parquet', '') for f in os.listdir(prices_dir) if f.endswith('.parquet')}

# 수집된 재무제표
collected_financials = set()
if os.path.exists(financials_dir):
    collected_financials = {d for d in os.listdir(financials_dir) if os.path.isdir(os.path.join(financials_dir, d))}

# 둘 다 있는 종목
fully_collected = collected_prices & collected_financials

print("=== 기존 수집 현황 ===")
print(f"가격 데이터 수집됨: {len(collected_prices)}개")
print(f"재무제표 수집됨: {len(collected_financials)}개")
print(f"완전히 수집됨: {len(fully_collected)}개")

# 수집 필요 종목
to_collect = [t for t in sp500 if t not in fully_collected]
print(f"\n수집 필요: {len(to_collect)}개")

=== 기존 수집 현황 ===
가격 데이터 수집됨: 502개
재무제표 수집됨: 503개
완전히 수집됨: 502개

수집 필요: 1개


## 5. 데이터 수집

In [25]:
# 배치 수집 실행 (수집 필요한 경우만)
if len(to_collect) == 0:
    print("✅ 모든 종목 이미 수집됨. 스킵합니다.")
    failed = []
    failed_reasons = {}
    elapsed = datetime.now() - datetime.now()  # 0초
else:
    success = 0
    failed = []
    failed_reasons = {}

    start_time = datetime.now()

    for ticker in tqdm(to_collect, desc="데이터 수집"):
        try:
            # 가격 데이터 (10년)
            prices = collector.collect_price_history(ticker, years=20)
            
            # 재무제표
            financials = collector.collect_financials(ticker)
            
            if prices is not None and financials is not None:
                success += 1
            else:
                failed.append(ticker)
                if prices is None:
                    failed_reasons[ticker] = "가격 데이터 없음"
                    print(f"가격 데이터 없음: {ticker}")
                else:
                    failed_reasons[ticker] = "재무제표 없음"
                
            # API 부하 방지
            time.sleep(0.5)
            
        except Exception as e:
            failed.append(ticker)
            failed_reasons[ticker] = str(e)

    elapsed = datetime.now() - start_time

    print(f"\n" + "="*50)
    print(f"✅ 수집 완료")
    print(f"="*50)
    print(f"성공: {success}/{len(to_collect)}")
    print(f"실패: {len(failed)}개")
    print(f"소요 시간: {elapsed}")

데이터 수집:   0%|          | 0/1 [00:00<?, ?it/s]HTTP Error 404: {"quoteSummary":{"result":null,"error":{"code":"Not Found","description":"Quote not found for symbol: WBA"}}}
$WBA: possibly delisted; no timezone found


가격 데이터 없음: WBA


HTTP Error 404: {"quoteSummary":{"result":null,"error":{"code":"Not Found","description":"Quote not found for symbol: WBA"}}}


가격 데이터 없음: WBA


데이터 수집: 100%|██████████| 1/1 [00:01<00:00,  1.91s/it]


✅ 수집 완료
성공: 0/1
실패: 1개
소요 시간: 0:00:01.916756


In [26]:
# 실패 종목 상세
if failed:
    print("=== 실패 종목 ===")
    for ticker in failed[:30]:  # 처음 30개만
        reason = failed_reasons.get(ticker, "알 수 없음")
        print(f"  {ticker}: {reason}")
    
    if len(failed) > 30:
        print(f"  ... 외 {len(failed) - 30}개")

=== 실패 종목 ===
  WBA: 가격 데이터 없음


## 6. 수집 결과 검증

In [27]:
# 최종 수집 현황
collected_prices = set()
if os.path.exists(prices_dir):
    collected_prices = {f.replace('.parquet', '') for f in os.listdir(prices_dir) if f.endswith('.parquet')}

collected_financials = set()
if os.path.exists(financials_dir):
    collected_financials = {d for d in os.listdir(financials_dir) if os.path.isdir(os.path.join(financials_dir, d))}

fully_collected = collected_prices & collected_financials

print("=== 최종 수집 현황 ===")
print(f"가격 데이터: {len(collected_prices)}개")
print(f"재무제표: {len(collected_financials)}개")
print(f"완전 수집: {len(fully_collected)}개")
print(f"\nS&P 500 커버리지: {len(fully_collected & set(sp500))}/{len(sp500)} ({100*len(fully_collected & set(sp500))/len(sp500):.1f}%)")

=== 최종 수집 현황 ===
가격 데이터: 502개
재무제표: 503개
완전 수집: 502개

S&P 500 커버리지: 502/503 (99.8%)


In [28]:
# 샘플 데이터 확인
sample_ticker = list(fully_collected)[0]

print(f"=== {sample_ticker} 샘플 데이터 ===")

# 가격 데이터
prices = collector.load_price_data(sample_ticker)
print(f"\n📈 가격 데이터: {prices.shape}")
print(f"   기간: {prices['Date'].min()} ~ {prices['Date'].max()}")

# 재무제표
financials = collector.load_financials(sample_ticker)
print(f"\n📊 재무제표:")
for key, val in financials.items():
    if isinstance(val, pd.DataFrame):
        print(f"   {key}: {val.shape}")
    elif isinstance(val, dict):
        print(f"   {key}: {len(val)} items")

=== ITW 샘플 데이터 ===

📈 가격 데이터: (5027, 7)
   기간: 2006-01-03 00:00:00 ~ 2025-12-24 00:00:00

📊 재무제표:
   balance_sheet_quarterly: (7, 83)
   balance_sheet_annual: (5, 85)
   income_stmt_annual: (4, 49)
   cash_flow_annual: (5, 66)
   info: 182 items
   income_stmt_quarterly: (5, 41)
   cash_flow_quarterly: (7, 64)


## 7. 수집 결과 저장

In [29]:
# 수집 로그 저장 (5X Finder로)
collection_log = {
    'timestamp': datetime.now().isoformat(),
    'version': 'v2',
    'total_tickers': len(sp500),
    'collected': len(fully_collected),
    'failed': failed,
    'failed_reasons': failed_reasons,
    'elapsed_seconds': elapsed.total_seconds() if elapsed else 0
}

log_path = os.path.join(output_dir, 'collection_log.json')
with open(log_path, 'w') as f:
    json.dump(collection_log, f, indent=2, default=str)

print(f"✅ 수집 로그 저장: {log_path}")

✅ 수집 로그 저장: /Users/kuka/5X Finder/data/raw/collection_log.json


In [30]:
# 종목별 데이터 시작 연도 저장 (5X Finder로)
start_years_output = os.path.join(output_dir, 'ticker_start_years.json')
with open(start_years_output, 'w') as f:
    json.dump(ticker_start_years, f, indent=2)

print(f"✅ 종목별 시작 연도 저장: {start_years_output}")
print(f"   총 {len(ticker_start_years)}개 종목")

✅ 종목별 시작 연도 저장: /Users/kuka/5X Finder/data/raw/ticker_start_years.json
   총 502개 종목


## 8. 요약

In [ ]:
print("="*50)
print("📋 Data Collection 완료")
print("="*50)

print(f"\n📊 수집 현황")
print(f"  S&P 500 커버리지: {len(fully_collected)}/{len(sp500)} ({100*len(fully_collected)/len(sp500):.1f}%)")
print(f"  실패 종목: {len(failed)}개")

print(f"\n📁 저장된 파일 ({output_dir})")
print(f"  - collection_log.json (수집 로그)")
print(f"  - ticker_start_years.json (동적 필3터링용)")

print(f"\n🔜 다음 단계")
print(f"  03_feature_engineering.ipynb - Feature 계산 (동적 필터링)")

📋 Data Collection 완료

📊 수집 현황
  S&P 500 커버리지: 502/503 (99.8%)
  실패 종목: 1개

📁 저장된 파일 (/Users/kuka/5X Finder/data/raw)
  - collection_log.json (수집 로그)
  - ticker_start_years.json (동적 필터링용)

🔜 다음 단계
  03_feature_engineering.ipynb - Feature 계산 (동적 필터링)
